In [2]:
import arcpy, sys
import os


In [50]:

# 设置工作环境和输入文件夹路径
input_folder = r'D:\lakemapping\10_paper_writting\figure\sample\sampleV8_test_region'
output_folder = r'D:\lakemapping\10_paper_writting\figure\sample\sampleV8_test_region_colormap'

# 设置色带范围和颜色
min_value = -100
max_value = 100

# 获取文件夹中的所有tif影像文件
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith('tif'):
            print(file)
            raster_path=os.path.join(input_folder,file)
            raster_layer = arcpy.management.MakeRasterLayer(raster_path, file)
            layer=raster_layer.getOutput(0)
            symbology = layer.symbology
            symbology.updateColorizer('RasterUniqueValueColorizer')
            arcpy.management.BuildRasterAttributeTable(layer)
            symbology.colorizer.field = 'Value'
            for grp in symbology.colorizer.groups:
                for itm in grp.items:
                    label=int(itm.label)
                    if label>=0:
                        itm.color = {'RGB': [255-label*2.55, 255-label*2.55, 255, 100]}
                    if label <0:
                        itm.color = {'RGB': [255, 255+label*2.55, 255+label*2.55, 100]}
        # # #         应用符号化到栅格图层
            layer.symbology=symbology
#             print(symbology)
        #         arcpy.management.ApplySymbologyFromLayer(raster_layer, symbology)


ndwi_10m_type0_1565.tif


In [7]:
## SELECT PLD、GLAKES within test region

In [6]:
auxiliary_dataset_gdb=r'J:\lakemapping\auxiliary_dataset.gdb'
GLAKES=r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES\GLAKES.gdb\GLAKES'
# arcpy.MakeFeatureLayer_management(GLAKES,'GLAKES')

In [46]:
GLAKES_new_gdb=r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES_after_30p_land_mask.gdb'
GLAKES_iwSHIFT_afm_keep_arid=os.path.join(GLAKES_new_gdb,'au2_GLAKES_iwSHIFT_after_mask_gt3ha_keep_arid')
GLAKES_niwSHIFT=os.path.join(GLAKES_new_gdb,'au1_GLAKES_niwSHIFT')
GLAKES_iwSHIFT_afm_keep_arid_lyr='GLAKES_iwSHIFT_after_mask_gt3ha_keep_arid'
GLAKES_niwSHIFT_lyr='au1_GLAKES_niwSHIFT'

In [7]:
osm_reservoir=os.path.join(auxiliary_dataset_gdb,'osm_natural_water_reservoir')
arcpy.MakeFeatureLayer_management(osm_reservoir,'osm_reservoir')

GeoDAR=r'D:\lakemapping\0_auxiliary_data\GeoDAR\GeoDAR_v10_v11\GeoDAR_v11_reservoirs.shp'
arcpy.MakeFeatureLayer_management(GeoDAR,'GeoDAR')

<Result 'GeoDAR'>

In [47]:
arcpy.MakeFeatureLayer_management(GLAKES_iwSHIFT_afm_keep_arid,GLAKES_iwSHIFT_afm_keep_arid_lyr)
arcpy.MakeFeatureLayer_management(GLAKES_niwSHIFT,GLAKES_niwSHIFT_lyr)

<Result 'au1_GLAKES_niwSHIFT'>

In [49]:
base_dir=r'D:\lakemapping\4_prediction\model0130\sampleV8'
sample_result=os.path.join(base_dir,'GLAKES_new_test_lakes.shp')
sample_result=os.path.join(base_dir,'GLAKES_new.shp')
selectByLocation(GLAKES_niwSHIFT_lyr, 'INTERSECT', 'test_region','NEW_SELECTION')
selectByLocation(GLAKES_iwSHIFT_afm_keep_arid_lyr, 'INTERSECT', 'test_region','NEW_SELECTION')
arcpy.Merge_management([GLAKES_niwSHIFT_lyr,GLAKES_iwSHIFT_afm_keep_arid_lyr],sample_result)

<Result 'D:\\lakemapping\\4_prediction\\model0130\\sampleV8\\GLAKES_new.shp'>

In [ ]:
PLD_list=['af','eu','si','as','au','sa','na','ar','gr']
new_PLD_gdb=r'J:\lakemapping\PLD\PLD.gdb'

In [ ]:
merge_list=[]
base_dir=r'D:\lakemapping\4_prediction\model0130\sampleV8'
sample_result=os.path.join(base_dir,'PLD_test_lakes.shp')
for region in PLD_list:
    print(region)
    PLD_region=os.path.join(new_PLD_gdb,f'PLD_{region}')
    PLD_region_lyr=f'PLD_{region}'
#     arcpy.MakeFeatureLayer_management(PLD_region,PLD_region_lyr)
    selectByLocation(PLD_region_lyr, 'INTERSECT', 'test_region','NEW_SELECTION')
    merge_list.append(PLD_region_lyr)
arcpy.Merge_management(merge_list,sample_result)

In [53]:
merge_list=[]
version='v15_241128'
output_dir = os.path.join(r'J:\lakemapping\postprocess',version)
merge_gdb=os.path.join(output_dir,f'5_polygon_afm_mergeWith_GLAKES_PLD.gdb')
sample_result=os.path.join(base_dir,'Hu_lakes')
for i in range(0,8):
    continent=eight_continents[i]
    c_lake_afm_mw_GP=os.path.join(merge_gdb,f'c{i+1}_{continent}_lakes')
    c_lake_afm_mw_GP_lyr=f'c{i+1}_{continent}_lakes'
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP,c_lake_afm_mw_GP_lyr)
    selectByLocation(c_lake_afm_mw_GP_lyr, 'INTERSECT', 'test_region','NEW_SELECTION')
    merge_list.append(c_lake_afm_mw_GP_lyr)
arcpy.Merge_management(merge_list,sample_result)

<Result 'D:\\lakemapping\\4_prediction\\model0130\\sampleV8\\Hu_lakes.shp'>

In [12]:
base_dir = r'D:\lakemapping\4_prediction\model0130\sampleV8'
iew='100'
version = r'test_buffer_0_02degree\result\iew'+iew+'\shp_result'
print(version)
predicted_polygon_dir = os.path.join(base_dir,version)
predicted_polygons_SJ = os.path.join(predicted_polygon_dir , f'iew{iew}_predicted_polygons_SJ.shp')
predicted_polygons_SJ_lyr = 'predicted_polygons_SJ'

test_buffer_0_02degree\result\iew100\shp_result


In [35]:
def selectByLocation(inputFeature,overlapType,selectionFeature,selectionType='NEW_SELECTION'):
    arcpy.SelectLayerByLocation_management(inputFeature,overlapType,selectionFeature,selection_type=selectionType)
    
def selectByAttribute(inputFeature,selectionType='NEW_SELECTION',code=''):
    arcpy.management.SelectLayerByAttribute(inputFeature,selectionType,code)
    
def calculateOperateByAttribute(inputFeature,code, column,value):
    selectByAttribute(inputFeature,'NEW_SELECTION',code)
    arcpy.CalculateField_management(inputFeature, column,value)
    
def calculateOperateByLocation(inputFeature,overlapType,selectionFeature,column,value,selectionType='NEW_SELECTION'):
    selectByLocation(inputFeature,overlapType,selectionFeature,selectionType)
    arcpy.CalculateField_management(inputFeature, column,value)

In [30]:
# arcpy.MakeFeatureLayer_management(predicted_polygons_SJ,predicted_polygons_SJ_lyr)
arcpy.AddField_management(predicted_polygons_SJ_lyr, 'operate', "Double")

<Result 'predicted_polygons_SJ'>

In [29]:
arcpy.AddField_management(predicted_polygons_SJ_lyr, 'flag', "Double")
selectByAttribute('GLAKES_new',code='Area_bound <=1 and Rser_flag =0')
calculateOperateByLocation(predicted_polygons_SJ_lyr,'INTERSECT','GLAKES_new','flag',0.5)
selectByAttribute('GLAKES_new',code='Area_bound >=1 and Rser_flag =0')
calculateOperateByLocation(predicted_polygons_SJ_lyr,'INTERSECT','GLAKES_new','flag',1)
selectByAttribute('GLAKES_new',code='Rser_flag =1')
calculateOperateByLocation(predicted_polygons_SJ_lyr,'INTERSECT','GLAKES_new','flag',2)
selectByAttribute(predicted_polygons_SJ_lyr,'NEW_SELECTION', 'flag IS NULL OR "flag" <1 ')
calculateOperateByLocation(predicted_polygons_SJ_lyr,'INTERSECT','GeoDAR','flag',3,'SUBSET_SELECTION')
selectByAttribute(predicted_polygons_SJ_lyr,'NEW_SELECTION', 'flag IS NULL OR "flag" <1 ')
calculateOperateByLocation(predicted_polygons_SJ_lyr,'INTERSECT','osm_reservoir','flag',4,'SUBSET_SELECTION')

In [54]:
#iwG aGm:flag=0.5、1、2 or 部分3、4 area ratio<1
#保留与水库相交的湖泊
calculateOperateByAttribute(predicted_polygons_SJ_lyr, '"area_ratio" >=1', 'operate',1)
# calculateOperateByAttribute(predicted_polygons_SJ_lyr,'"flag" >=2','operate',1)
#和GLAKES小自然湖相交，面积大于1，ar>0.9 保留；和GLAKES大自然湖相交，ar>0.8保留
# calculateOperateByAttribute(predicted_polygons_SJ_lyr,'"P_area">=1 and "area_ratio" >= 0.9', 'operate',1)#iwG niwG都满足
# calculateOperateByAttribute(predicted_polygons_SJ_lyr,'"flag" = 1 AND "area_ratio" >= 0.8', 'operate',1)
# #其他，保留mask后 本来应该operate IS null
# calculateOperateByAttribute(predicted_polygons_SJ_lyr,'operate IS NULL OR operate =0','operate',0.5)#flag =0.5 OR 1
#和GLAKES小自然湖相交，AR<0.1,删除
# calculateOperateByAttribute(predicted_polygons_SJ_lyr,'"flag" = 0.5 AND "area_ratio" < 0.1', 'operate',0)
# #niwG flag=0 or 部分3、4
# #不和其他数据相交，AR<0.8,删除
# calculateOperateByAttribute(predicted_polygons_SJ_lyr,'"flag" = 0 AND "area_ratio" < 0.8', 'operate',0)

In [51]:
lakes_afterMask=os.path.join(predicted_polygon_dir , f'iew{iew}_predicted_polygons_SJ_afm.shp')
lakes_afterMask_lyr='lakes_afterMask'
selectByAttribute(predicted_polygons_SJ_lyr,'NEW_SELECTION','"operate" = 0.5')
# selectByLocation('iew100_predicted_polygons_afm_point', 'WITHIN', predicted_polygons_SJ_lyr,'NEW_SELECTION')
selectByLocation('iew100_predicted_polygons_afm', 'WITHIN', predicted_polygons_SJ_lyr,'NEW_SELECTION')
# selectByAttribute('iew100_predicted_polygons_afm_point','SUBSET_SELECTION','"operate" IS NULL')
# selectByAttribute('iew100_predicted_polygons_afm_point','ADD_TO_SELECTION','"operate" = 1')
selectByAttribute(predicted_polygons_SJ_lyr,'NEW_SELECTION','"operate" >= 1')

arcpy.Merge_management([predicted_polygons_SJ_lyr,'iew100_predicted_polygons_afm'],lakes_afterMask)
arcpy.MakeFeatureLayer_management(lakes_afterMask,lakes_afterMask_lyr)
selectByAttribute(lakes_afterMask_lyr,'NEW_SELECTION','P_area IS NULL')
arcpy.CalculateField_management(lakes_afterMask_lyr, "P_area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")

<Result 'lakes_afterMask'>

In [ ]:
operate = 0.5 And area_ratio < 0.01 Or operate = 0